### Note
* Instructions have been included for each segment. You do not have to follow them exactly, but they are included to help you think through the steps.

In [1]:
# Dependencies and Setup
import pandas as pd

# File to Load (Remember to Change These)
school_data_to_load = "Resources/schools_complete.csv"
student_data_to_load = "Resources/students_complete.csv"

# Read School and Student Data File and store into Pandas DataFrames
school_data = pd.read_csv(school_data_to_load)
student_data = pd.read_csv(student_data_to_load)

# Combine the data into a single dataset.  
school_data_complete = pd.merge(student_data, school_data, how="left", on=["school_name", "school_name"])
school_data_complete.head()

,Student ID,student_name,gender,grade,school_name,reading_score,math_score,School ID,type,size,budget
0,0,Paul Bradley,M,9th,Huang High School,66,79,0,District,2917,1910635
1,1,Victor Smith,M,12th,Huang High School,94,61,0,District,2917,1910635
2,2,Kevin Rodriguez,M,12th,Huang High School,90,60,0,District,2917,1910635
3,3,Dr. Richard Scott,M,12th,Huang High School,67,58,0,District,2917,1910635
4,4,Bonnie Ray,F,9th,Huang High School,97,84,0,District,2917,1910635


## District Summary

* Calculate the total number of schools

* Calculate the total number of students

* Calculate the total budget

* Calculate the average math score 

* Calculate the average reading score

* Calculate the percentage of students with a passing math score (70 or greater)

* Calculate the percentage of students with a passing reading score (70 or greater)

* Calculate the percentage of students who passed math **and** reading (% Overall Passing)

* Create a dataframe to hold the above results

* Optional: give the displayed data cleaner formatting

In [2]:
# Calculate the total number of schools
schoolsCount = school_data_complete["School ID"].value_counts()

# Calculate the total number of students
studentsCount = len(student_data)

# Calculate the total budget
budgetSum = school_data["budget"].sum()

# Calculate the average math score 
mathScoreMean = student_data["math_score"].mean()

# Calculate the average reading score
readingScoreMean = student_data["reading_score"].mean()

# Calculate the percentage of students with a passing math score (70 or greater)
mathPassedSum = student_data.loc[student_data["math_score"] >= 70]
mathPassedPercentage = len(mathPassedSum) / studentsCount * 100

# Calculate the percentage of students with a passing reading score (70 or greater)
readingPassedSum = student_data.loc[student_data["reading_score"] >= 70]
readingPassedPercentage = len(readingPassedSum) / studentsCount * 100

bothPassedSum = student_data.loc[(student_data["reading_score"] >= 70) & (student_data["math_score"] >= 70)]
bothPassedPercentage = len(bothPassedSum) / studentsCount * 100

# Create a dataframe to hold the above results

summaryDict = [{ "Total Schools" : len(schoolsCount),
               "Total Students" : studentsCount,
               "Total Budget" : budgetSum,
               "Average Math Score" : mathScoreMean,
               "Average Reading Score" : readingScoreMean,
               "% Passing Math" : mathPassedPercentage,
               "% Passing Reading" : readingPassedPercentage,
               "% Overall Passing" : bothPassedPercentage,
               }]
summary = pd.DataFrame(summaryDict)

# Optional: give the displayed data cleaner formatting
summary["Total Students"] = summary["Total Students"].map("{:,}".format)
summary["Total Budget"] = summary["Total Budget"].map("${:,.2f}".format)
summary


,Total Schools,Total Students,Total Budget,Average Math Score,Average Reading Score,% Passing Math,% Passing Reading,% Overall Passing
0,15,"39,170","$24,649,428.00",78.985371,81.87784,74.980853,85.805463,65.172326


## School Summary

* Create an overview table that summarizes key metrics about each school, including:
  * School Name
  * School Type
  * Total Students
  * Total School Budget
  * Per Student Budget
  * Average Math Score
  * Average Reading Score
  * % Passing Math
  * % Passing Reading
  * % Overall Passing (The percentage of students that passed math **and** reading.)
  
* Create a dataframe to hold the above results

In [3]:
# we're taking select columns from our excel files, doing math, and moving them into 'mypd' which will be the resulting end-graph
# get relevant columns and sort so row order stays consistent
mypd = school_data[["school_name","type","size","budget"]].sort_values("school_name")

# pretty up column names
cols = { "school_name" : "School Name",
            "type" :  "School Type",
            "size" : "Total Students",
            "budget" : "Total School Budget"}
mypd = mypd.rename(columns=cols)

# calculate Per Student Budget
mypd["Per Student Budget"] = mypd["Total School Budget"] / mypd["Total Students"]

# group our students per school, get mean and counts
sps = school_data_complete[["school_name", "math_score", "reading_score"]].sort_values("school_name")
spsGrouped = sps.groupby("school_name")
spsGroupedMean = spsGrouped.mean()

# move summary data from sps to mypd using tolist()
mypd["Average Math Score"] = spsGroupedMean["math_score"].tolist()
mypd["Average Reading Score"] = spsGroupedMean["reading_score"].tolist()

# who passed math? doing this on separate dataframes where we can manipulate and later add to mypd
passedMath = sps.loc[(sps["math_score"]>=70),["school_name", "math_score"]]
# who passed reading? 
passedReading = sps.loc[(sps["reading_score"]>=70),["school_name", "reading_score"]]
# lets summarize those who passed
passedGrouped = passedMath.groupby("school_name").count()
passedGrouped["reading_score"] = passedReading.groupby("school_name").count()
passedGrouped["total_students"] = mypd["Total Students"].tolist()
# percentage math
passedGrouped["passed_math"] = passedGrouped["math_score"] / passedGrouped["total_students"] * 100
passedGrouped["passed_reading"] = passedGrouped["reading_score"] / passedGrouped["total_students"] * 100
# who passed both math AND reading?
passedBoth = sps.loc[(sps["reading_score"] >= 70) & (sps["math_score"] >= 70),:]

passedGrouped["sum_both"] = passedBoth.groupby("school_name").count()["reading_score"].tolist() 
passedGrouped["passed_both"] = passedGrouped["sum_both"] / passedGrouped["total_students"] * 100

# results look good, lets move columns we're after to mypd
mypd["% Passing Math"] =  passedGrouped["passed_math"].tolist()
mypd["% Passing Reading"] =  passedGrouped["passed_reading"].tolist()
mypd["% Overall Passing"] =  passedGrouped["passed_both"].tolist()

# more formatting
mypd["Total School Budget"] = mypd["Total School Budget"].map("${:,.2f}".format)
mypd["Per Student Budget"] = mypd["Per Student Budget"].map("${:,.2f}".format)
mypd.set_index("School Name",inplace=True)
  
mypd

,School Type,Total Students,Total School Budget,Per Student Budget,Average Math Score,Average Reading Score,% Passing Math,% Passing Reading,% Overall Passing
School Name,,,,,,,,,
Bailey High School,District,4976,"$3,124,928.00",$628.00,77.048432,81.033963,66.680064,81.933280,54.642283
Cabrera High School,Charter,1858,"$1,081,356.00",$582.00,83.061895,83.975780,94.133477,97.039828,91.334769
Figueroa High School,District,2949,"$1,884,411.00",$639.00,76.711767,81.158020,65.988471,80.739234,53.204476
Ford High School,District,2739,"$1,763,916.00",$644.00,77.102592,80.746258,68.309602,79.299014,54.289887
Griffin High School,Charter,1468,"$917,500.00",$625.00,83.351499,83.816757,93.392371,97.138965,90.599455
Hernandez High School,District,4635,"$3,022,020.00",$652.00,77.289752,80.934412,66.752967,80.862999,53.527508
Holden High School,Charter,427,"$248,087.00",$581.00,83.803279,83.814988,92.505855,96.252927,89.227166
Huang High School,District,2917,"$1,910,635.00",$655.00,76.629414,81.182722,65.683922,81.316421,53.513884
Johnson High School,District,4761,"$3,094,650.00",$650.00,77.072464,80.966394,66.057551,81.222432,53.539172


In [4]:
## Top Performing Schools (By % Overall Passing

mypd_sorted = mypd.sort_values("% Overall Passing", ascending=False).head()
mypd_sorted

,School Type,Total Students,Total School Budget,Per Student Budget,Average Math Score,Average Reading Score,% Passing Math,% Passing Reading,% Overall Passing
School Name,,,,,,,,,
Cabrera High School,Charter,1858,"$1,081,356.00",$582.00,83.061895,83.975780,94.133477,97.039828,91.334769
Thomas High School,Charter,1635,"$1,043,130.00",$638.00,83.418349,83.848930,93.272171,97.308869,90.948012
Griffin High School,Charter,1468,"$917,500.00",$625.00,83.351499,83.816757,93.392371,97.138965,90.599455
Wilson High School,Charter,2283,"$1,319,574.00",$578.00,83.274201,83.989488,93.867718,96.539641,90.582567
Pena High School,Charter,962,"$585,858.00",$609.00,83.839917,84.044699,94.594595,95.945946,90.540541


## Bottom Performing Schools (By % Overall Passing)

* Sort and display the five worst-performing schools by % overall passing.

In [11]:
mypd_sorted_asc = mypd.sort_values("% Overall Passing", ascending=True).head()
mypd_sorted_asc

,School Type,Total Students,Total School Budget,Per Student Budget,Average Math Score,Average Reading Score,% Passing Math,% Passing Reading,% Overall Passing
Rodriguez High School,District,3999,"$2,547,363.00",$637.00,76.842711,80.744686,66.366592,80.220055,52.988247
Figueroa High School,District,2949,"$1,884,411.00",$639.00,76.711767,81.158020,65.988471,80.739234,53.204476
Huang High School,District,2917,"$1,910,635.00",$655.00,76.629414,81.182722,65.683922,81.316421,53.513884
Hernandez High School,District,4635,"$3,022,020.00",$652.00,77.289752,80.934412,66.752967,80.862999,53.527508
Johnson High School,District,4761,"$3,094,650.00",$650.00,77.072464,80.966394,66.057551,81.222432,53.539172


## Math Scores by Grade

* Create a table that lists the average Reading Score for students of each grade level (9th, 10th, 11th, 12th) at each school.

  * Create a pandas series for each grade. Hint: use a conditional statement.
  
  * Group each series by school
  
  * Combine the series into a dataframe
  
  * Optional: give the displayed data cleaner formatting

In [42]:
# sd = student_data.sort_values("school_name")
# sd.groupby(["school_name", "grade"]).count().head()

# # count9 = mathPassGroup9.count()["grade"]
# sum9 = mathPassGroup9.sum()
# sum9["reading_score"]

mathPassedSum9 = student_data.loc[student_data["grade"] == "9th"]
mathPassGroup9 = mathPassedSum9.groupby("school_name").mean()["math_score"]
mathPassedSum10 = student_data.loc[student_data["grade"] == "10th"]
mathPassGroup10 = mathPassedSum10.groupby("school_name").mean()["math_score"]
mathPassedSum11 = student_data.loc[student_data["grade"] == "11th"]
mathPassGroup11 = mathPassedSum11.groupby("school_name").mean()["math_score"]
mathPassedSum12 = student_data.loc[student_data["grade"] == "12th"]
mathPassGroup12 = mathPassedSum12.groupby("school_name").mean()["math_score"]

# mathPassGroup9.head()


byGradeDf = pd.DataFrame({"9th": mathPassGroup9,
                          "10th": mathPassGroup9,
                          "11th": mathPassGroup9,
                          "12th": mathPassGroup9
                          })

byGradeDf


# sd.head()
# schools = sd["school_name"].tolist()
# grades = sd["grade"].tolist()
# scores = sd["reading_score"].tolist()
# grade9 = []
# for score in scores:
    


,9th,10th,11th,12th
school_name,,,,
Bailey High School,77.083676,77.083676,77.083676,77.083676
Cabrera High School,83.094697,83.094697,83.094697,83.094697
Figueroa High School,76.403037,76.403037,76.403037,76.403037
Ford High School,77.361345,77.361345,77.361345,77.361345
Griffin High School,82.044010,82.044010,82.044010,82.044010
Hernandez High School,77.438495,77.438495,77.438495,77.438495
Holden High School,83.787402,83.787402,83.787402,83.787402
Huang High School,77.027251,77.027251,77.027251,77.027251
Johnson High School,77.187857,77.187857,77.187857,77.187857


## Reading Score by Grade 

* Perform the same operations as above for reading scores

In [43]:
readPassedSum9 = student_data.loc[student_data["grade"] == "9th"]
readPassGroup9 = readPassedSum9.groupby("school_name").mean()["reading_score"]
readPassedSum10 = student_data.loc[student_data["grade"] == "10th"]
readPassGroup10 = readPassedSum10.groupby("school_name").mean()["reading_score"]
readPassedSum11 = student_data.loc[student_data["grade"] == "11th"]
readPassGroup11 = readPassedSum11.groupby("school_name").mean()["reading_score"]
readPassedSum12 = student_data.loc[student_data["grade"] == "12th"]
readPassGroup12 = readPassedSum12.groupby("school_name").mean()["reading_score"]

# mathPassGroup9.head()


byGradeDf2 = pd.DataFrame({"9th": readPassGroup9,
                          "10th": readPassGroup9,
                          "11th": readPassGroup9,
                          "12th": readPassGroup9
                          })

byGradeDf2

,9th,10th,11th,12th
school_name,,,,
Bailey High School,81.303155,81.303155,81.303155,81.303155
Cabrera High School,83.676136,83.676136,83.676136,83.676136
Figueroa High School,81.198598,81.198598,81.198598,81.198598
Ford High School,80.632653,80.632653,80.632653,80.632653
Griffin High School,83.369193,83.369193,83.369193,83.369193
Hernandez High School,80.866860,80.866860,80.866860,80.866860
Holden High School,83.677165,83.677165,83.677165,83.677165
Huang High School,81.290284,81.290284,81.290284,81.290284
Johnson High School,81.260714,81.260714,81.260714,81.260714


## Scores by School Spending

* Create a table that breaks down school performances based on average Spending Ranges (Per Student). Use 4 reasonable bins to group school spending. Include in the table each of the following:
  * Average Math Score
  * Average Reading Score
  * % Passing Math
  * % Passing Reading
  * Overall Passing Rate (Average of the above two)

In [74]:
grouped_school_data = school_data_complete.groupby(["school_name", "type"])
studentsGroup = grouped_school_data["Student ID"].count()

# budgetGroup = grouped_school_data["budget"]
# totalBudgetGroup= total_budget_grp/ total_students_grp
budgetGroup = grouped_school_data["budget"].mean()
budgetPerStudent =  (budgetGroup/ studentsGroup)
# mathMean = school_data_complete["math_score"].mean()

# total_students_grp = grouped_school_data["Student ID"].count()
# # total_budget_grp = grouped_school_data["budget"]
# # total_budget_grp_var = total_budget_grp/ total_students_grp
# total_budget_grp = grouped_school_data["budget"].mean()
# per_stu_budget_grp =  (total_budget_grp/ total_students_grp)
mathMean = grouped_school_data["math_score"].mean()
readMean = grouped_school_data["reading_score"].mean()
#using query to get passing math scores
mathPass = school_data_complete.query('math_score >70')["School ID"].count()/ studentsGroup
readPass = school_data_complete.query('reading_score >70')["School ID"].count()/ studentsGroup 
overallPass = (mathMean + readMean)/2


bins=[0,585,615,645,675]
bin_names = ["< $585", "$585-615","$615-645","$645-675"]


# grouped_school_data.head()
grouped_school_data_df = pd.DataFrame({"Total Students":studentsGroup,
                                       "Total School Budget": budgetGroup, 
                                       "Per Student Budget":budgetPerStudent,
                                       "Average Math Score": mathMean,
                                       "Average Reading Score": readMean,   
                                       "% Passing Math":mathPass,
                                       "% Passing Reading":readPass,
                                        "%Overall Passing":overallPass          
                        
                                      
})
# grouped_school_data_df.head(10)
grouped_school_data_df["Spending ranges (Per Student)"] = pd.cut(per_stu_budget_grp, bins, labels=bin_names)

# newdf = pd.cut(per_stu_budget_grp, bins, labels=bin_names)
# newdf.head()
spendMath = grouped_school_data_df.groupby(["Spending ranges (Per Student)"]).mean()["Average Math Score"]
#spendMath, will likely reuse this for next question
spendRead = grouped_school_data_df.groupby(["Spending ranges (Per Student)"]).mean()["Average Reading Score"]
spendPassMath = grouped_school_data_df.groupby(["Spending ranges (Per Student)"]).mean()["% Passing Math"]
spendPassRead = grouped_school_data_df.groupby(["Spending ranges (Per Student)"]).mean()["% Passing Reading"]
spendOverall = (spendMath + spendRead)/2

spendDf = pd.DataFrame({"Average Math Score":spendMath,
                                    "Average Reading Score":spendRead,
                                    "% Passing Math":spendPassMath,
                                    "% Passing Reading":spendPassRead,
                                    "% Overall Passing": spendOverall})

spendDf

,Average Math Score,Average Reading Score,% Passing Math,% Passing Reading,% Overall Passing
Spending ranges (Per Student),,,,,
< $585,83.455399,83.933814,27.460725,31.473887,83.694607
$585-615,83.599686,83.885211,22.789153,26.119603,83.742449
$615-645,79.079225,81.891436,11.569445,13.260225,80.485330
$645-675,76.997210,81.027843,7.264879,8.326582,79.012526


## Scores by School Size

* Perform the same operations as above, based on school size.

In [68]:
# new bins reflecting template
bins = [0, 1000, 2000, 5000]
sizes = ["Small (<1000)", "Medium (1000-2000)", "Large (2000-5000)"]

# cut/bin
grouped_school_data_df["Scores by School Size"] = pd.cut(total_students_grp, bins, labels=sizes)

#average our groups
sizeMath = grouped_school_data_df.groupby(["Scores by School Size"]).mean()["Average Math Score"]
sizeRead = grouped_school_data_df.groupby(["Scores by School Size"]).mean()["Average Reading Score"]
sizeMathPass = grouped_school_data_df.groupby(["Scores by School Size"]).mean()["% Passing Math"]
sizeReadPass = grouped_school_data_df.groupby(["Scores by School Size"]).mean()["% Passing Reading"]
sizeOverall = (sizeMath + sizeRead)/2

#df
schoolSizeDf = pd.DataFrame({"Average Math Score":sizeMath,
                                    "Average Reading Score":sizeRead,
                                    "% Passing Math":sizeMathPass,
                                    "% Passing Reading":sizeReadPass,
                                    "% Overall Passing": sizeOverall})

schoolSizeDf

,Average Math Score,Average Reading Score,% Passing Math,% Passing Reading,% Overall Passing Rate
Scores by School Size,,,,,
Small (<1000),83.821598,83.929843,47.941793,54.948098,83.875721
Medium (1000-2000),83.374684,83.864438,16.755263,19.203909,83.619561
Large (2000-5000),77.746417,81.344493,8.371576,9.595014,79.545455


## Scores by School Type

* Perform the same operations as above, based on school type

In [72]:
# grouping by type exposes Charter, District
# get means for each score of interest
typeMath = grouped_school_data_df.groupby(["type"]).mean()["Average Math Score"]
typeRead = grouped_school_data_df.groupby(["type"]).mean()["Average Reading Score"]
typeMathPass = grouped_school_data_df.groupby(["type"]).mean()["% Passing Math"]
typeReadPass = grouped_school_data_df.groupby(["type"]).mean()["% Passing Reading"]

typeOverall = (typeMath + typeRead)/2

#pass data into new df
schoolTypeDf = pd.DataFrame({"Average Math Score":typeMath,
                                    "Average Reading Score":typeRead,
                                    "% Passing Math":typeMathPass,
                                    "% Passing Reading":typeReadPass,
                                    "% Overall Passing": typeOverall})

schoolTypeDf

,Average Math Score,Average Reading Score,% Passing Math,% Passing Reading,% Overall Passing
type,,,,,
Charter,83.473852,83.896421,24.010050,27.518925,83.685136
District,76.956733,80.966636,7.793158,8.932066,78.961685
